In [1]:
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans

from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import pickle

In [2]:
df = pd.read_csv('/content/Test Data.csv')
df.head()

,user_id,aov,category
0,43323,19115,Phones
1,43323,29309,Phones
2,43323,15293,Phones
3,43323,23548,Phones
4,43323,751,Fashion


In [3]:
print(df['category'].unique())


['Phones' 'Fashion' 'Consumer Durables' 'Back to School' 'Books'
 'Ereaders' 'Home Decor' 'Toys' 'Groceries' 'Board Games' 'Gaming' 'TVs'
 'Beauty Products' 'Kitchen cleaning Supplies' 'Laptops' 'Pet Supplies'
 'Fitness' 'Painiting Supplies']


In [4]:
y=df['user_id'].unique()
y.shape
y.max()

50089

In [5]:
df = pd.get_dummies(df, columns = ['category'])
df.head(10)

,user_id,aov,category_Back to School,category_Beauty Products,category_Board Games,category_Books,category_Consumer Durables,category_Ereaders,category_Fashion,category_Fitness,category_Gaming,category_Groceries,category_Home Decor,category_Kitchen cleaning Supplies,category_Laptops,category_Painiting Supplies,category_Pet Supplies,category_Phones,category_TVs,category_Toys
0,43323,19115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,43323,29309,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,43323,15293,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,43323,23548,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,43323,751,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,43323,1017,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,43323,41480,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,43323,419,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,43323,610,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,43323,642,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df=df.groupby(['user_id']).sum()

In [7]:
df['user_id'] = df.index
df

,aov,category_Back to School,category_Beauty Products,category_Board Games,category_Books,category_Consumer Durables,category_Ereaders,category_Fashion,category_Fitness,category_Gaming,category_Groceries,category_Home Decor,category_Kitchen cleaning Supplies,category_Laptops,category_Painiting Supplies,category_Pet Supplies,category_Phones,category_TVs,category_Toys,user_id
user_id,,,,,,,,,,,,,,,,,,,,
2,52447,0,0,0,0,0,1,2,1,0,0,0,0,0,0,0,2,0,0,2
3,69210,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,3,0,1,3
8,131773,1,0,0,0,1,0,2,1,0,0,4,0,1,0,0,3,0,2,8
10,22582,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,2,10
13,72993,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0,3,0,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49998,42823,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,2,0,0,49998
50037,1729,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,50037
50050,1667,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,50050


In [8]:
df.shape

(19981, 20)

In [9]:
newdf=df.copy()

In [10]:
del df['aov']
del df['user_id']

In [11]:
df.head()

,category_Back to School,category_Beauty Products,category_Board Games,category_Books,category_Consumer Durables,category_Ereaders,category_Fashion,category_Fitness,category_Gaming,category_Groceries,category_Home Decor,category_Kitchen cleaning Supplies,category_Laptops,category_Painiting Supplies,category_Pet Supplies,category_Phones,category_TVs,category_Toys
user_id,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,1,2,1,0,0,0,0,0,0,0,2,0,0
3,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,3,0,1
8,1,0,0,0,1,0,2,1,0,0,4,0,1,0,0,3,0,2
10,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,2
13,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0,3,0,1


In [12]:
df.columns = [i.replace('category_',"") for i in df.columns]

In [13]:
df.head()

,Back to School,Beauty Products,Board Games,Books,Consumer Durables,Ereaders,Fashion,Fitness,Gaming,Groceries,Home Decor,Kitchen cleaning Supplies,Laptops,Painiting Supplies,Pet Supplies,Phones,TVs,Toys
user_id,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,1,2,1,0,0,0,0,0,0,0,2,0,0
3,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,3,0,1
8,1,0,0,0,1,0,2,1,0,0,4,0,1,0,0,3,0,2
10,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,2
13,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0,3,0,1


In [14]:
Tops =pd.DataFrame(df.apply(lambda x:list(df.columns[np.array(x).argsort()[::-1][:3]]), axis=1).to_list(),  columns=['Top1', 'Top2', 'Top3'])


Tops

,Top1,Top2,Top3
0,Phones,Fashion,Fitness
1,Phones,Fashion,Toys
2,Home Decor,Phones,Toys
3,Toys,Phones,Fashion
4,Phones,Toys,Home Decor
...,...,...,...
19976,Phones,Fashion,Home Decor
19977,Groceries,Toys,Fitness
19978,Back to School,Groceries,Fitness
19979,Kitchen cleaning Supplies,Toys,Fitness


In [15]:
Tops['category']=Tops.apply(lambda x:'%s,%s,%s' % (x['Top1'],x['Top2'],x['Top3']),axis=1)

In [16]:
Tops

,Top1,Top2,Top3,category
0,Phones,Fashion,Fitness,"Phones,Fashion,Fitness"
1,Phones,Fashion,Toys,"Phones,Fashion,Toys"
2,Home Decor,Phones,Toys,"Home Decor,Phones,Toys"
3,Toys,Phones,Fashion,"Toys,Phones,Fashion"
4,Phones,Toys,Home Decor,"Phones,Toys,Home Decor"
...,...,...,...,...
19976,Phones,Fashion,Home Decor,"Phones,Fashion,Home Decor"
19977,Groceries,Toys,Fitness,"Groceries,Toys,Fitness"
19978,Back to School,Groceries,Fitness,"Back to School,Groceries,Fitness"
19979,Kitchen cleaning Supplies,Toys,Fitness,"Kitchen cleaning Supplies,Toys,Fitness"


In [17]:
del Tops['Top1']
del Tops['Top2']
del Tops['Top3']

In [18]:
Tops

,category
0,"Phones,Fashion,Fitness"
1,"Phones,Fashion,Toys"
2,"Home Decor,Phones,Toys"
3,"Toys,Phones,Fashion"
4,"Phones,Toys,Home Decor"
...,...
19976,"Phones,Fashion,Home Decor"
19977,"Groceries,Toys,Fitness"
19978,"Back to School,Groceries,Fitness"
19979,"Kitchen cleaning Supplies,Toys,Fitness"


In [20]:
datatoexcel = pd.ExcelWriter('newdf.xlsx')
  
# write DataFrame to excel
newdf.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()

In [21]:
datatoexcel = pd.ExcelWriter('Tops.xlsx')
  
# write DataFrame to excel
Tops.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()